Neural Network (Basic sequential CNN)

In [ ]:
# Mount Google Drive if using Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [ ]:

# Define the paths to your image folders
motion_blur_path = "/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/motion_blurred"
defocused_blur_path = "/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/defocused_blurred"
sharp_images_path = "/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/sharp"


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


# Define the paths to your image folders
motion_blur_path = "/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/motion_blurred"
defocused_blur_path = "/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/defocused_blurred"
sharp_images_path = "/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/sharp"


# Define batch size and image size
batch_size = 32
img_size = (224, 224)  # You can adjust this based on your model's input size

# Create an ImageDataGenerator for loading and augmenting images
datagen = ImageDataGenerator(
    rescale=1./255,         # Normalize pixel values to [0, 1]
    validation_split=0.2,   # Reserve 20% of data for validation
    horizontal_flip=True,   # Augmentation: flip images horizontally
    rotation_range=20,      # Augmentation: rotate images
    zoom_range=0.2          # Augmentation: zoom in and out
)

# Load the images from the directory, creating training and validation sets
train_data = datagen.flow_from_directory(
    directory="/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # One-hot encoding for multi-class classification
    subset='training'          # Training data
)

val_data = datagen.flow_from_directory(
    directory="/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # One-hot encoding for multi-class classification
    subset='validation'        # Validation data
)

# Print the class indices to ensure correct labeling
print("Class indices:", train_data.class_indices)

# Automatically detect the number of classes
num_classes = len(train_data.class_indices)

# Build the CNN model
model = models.Sequential()

# Add convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output of the convolutional layers before passing it to fully connected layers
model.add(layers.Flatten())

# Fully connected (dense) layers
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))  # Add dropout for regularization to prevent overfitting

# Output layer: use softmax activation for multi-class classification
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()

# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,   # Adjust the number of epochs based on your needs
    steps_per_epoch=train_data.samples // batch_size,
    validation_steps=val_data.samples // batch_size
)

# Save the trained model to a file
model.save('blur_classification_model.h5')

# Evaluate the model on the validation data
loss, accuracy = model.evaluate(val_data)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


Found 840 images belonging to 3 classes.
Found 210 images belonging to 3 classes.
Class indices: {'defocused_blurred': 0, 'motion_blurred': 1, 'sharp': 2}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 24, 24, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 18432)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 512)                 │       9,437,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,680,067 (36.93 MB)

...

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 331s 10s/step - accuracy: 0.3597 - loss: 1.2453 - val_accuracy: 0.5104 - val_loss: 0.9911
Epoch 2/10
 1/26 ━━━━━━━━━━━━━━━━━━━━ 1:50 4s/step - accuracy: 0.5625 - loss: 0.9857

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.5625 - loss: 0.9857 - val_accuracy: 0.3889 - val_loss: 1.0588
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 204s 6s/step - accuracy: 0.4905 - loss: 0.9553 - val_accuracy: 0.5521 - val_loss: 0.9535
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.5312 - loss: 0.7978 - val_accuracy: 0.3333 - val_loss: 1.7251
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 201s 6s/step - accuracy: 0.5349 - loss: 0.8770 - val_accuracy: 0.4844 - val_loss: 0.9555
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.5938 - loss: 0.8707 - val_accuracy: 0.7778 - val_loss: 1.1417
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 199s 6s/step - accuracy: 0.5234 - loss: 0.9077 - val_accuracy: 0.5156 - val_loss: 1.1480
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.5625 - loss: 0.8900 - val_accuracy: 0.6667 - val_loss: 1.0796
Epoch 9/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 154s 5s/step - accuracy: 0.5569 - loss: 0.8006 - val_accuracy: 0.6042 - val_loss: 0.

7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.5550 - loss: 0.9499
Validation Accuracy: 57.14%


Neural Network (Simplified-Fast-Alexnet)

In [ ]:
# Mount Google Drive if using Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import shutil

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to the image directories
base_dir = '/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/'  # Update with your directory containing 3 folders (sharp, motion_blurred, defocused_blurred)

# Image parameters
img_height, img_width = 227, 227
batch_size = 32
epochs = 10  # You can adjust this

# Data Preprocessing - Image Augmentation and Normalization with validation split
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split for testing
)

# Load training data directly from base_dir, using subset='training' for training
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Training data
)

# Load validation data directly from base_dir, using subset='validation' for validation
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Validation data
)

# Simplified Fast AlexNet (SFA) Model
def create_sfa_model(input_shape=(227, 227, 3), num_classes=3):
    model = models.Sequential()

    # Conv Layer 1
    model.add(layers.Conv2D(48, (11, 11), strides=4, input_shape=input_shape, padding='valid'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

    # Conv Layer 2
    model.add(layers.Conv2D(128, (5, 5), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

    # Conv Layer 3
    model.add(layers.Conv2D(192, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))

    # Conv Layer 4
    model.add(layers.Conv2D(192, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))

    # Conv Layer 5
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

    # Fully Connected Layer
    model.add(layers.Flatten())
    model.add(layers.Dense(128))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))

    # Output Layer (Softmax for classification)
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Create the SFA model
model = create_sfa_model()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model Summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

# Evaluate the model on the validation set
test_loss, test_acc = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f'Validation accuracy: {test_acc * 100:.2f}%')


Found 840 images belonging to 3 classes.
Found 210 images belonging to 3 classes.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)                   │ (None, 55, 55, 48)          │          17,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 55, 55, 48)          │             192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_12 (Activation)           │ (None, 55, 55, 48)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 27, 27, 48)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 27, 27, 128)         │         153,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 27, 27, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_13 (Activation)           │ (None, 27, 27, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 13, 13, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 13, 13, 192)         │         221,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 13, 13, 192)         │             768 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_14 (Activation)           │ (None, 13, 13, 192)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 13, 13, 192)         │         331,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 13, 13, 192)         │             768 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_15 (Activation)           │ (None, 13, 13, 192)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 13, 13, 128)         │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 13, 13, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_16 (Activation)           │ (None, 13, 13, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 4608)                │              

 Total params: 1,539,459 (5.87 MB)

 Trainable params: 1,537,827 (5.87 MB)

 Non-trainable params: 1,632 (6.38 KB)

Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step - accuracy: 0.3773 - loss: 1.3158 - val_accuracy: 0.3385 - val_loss: 98.0183
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.2812 - loss: 1.2115 - val_accuracy: 0.3889 - val_loss: 42.2498
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 84s 3s/step - accuracy: 0.3730 - loss: 1.1690 - val_accuracy: 0.3385 - val_loss: 65.0731
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.2812 - loss: 1.2454 - val_accuracy: 0.2778 - val_loss: 67.8305
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 140s 3s/step - accuracy: 0.4159 - loss: 1.1238 - val_accuracy: 0.3542 - val_loss: 6.7550
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.2188 - loss: 1.1937 - val_accuracy: 0.5000 - val_loss: 4.7121
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.4231 - loss: 1.0777 - val_accuracy: 0.3125 - val_loss: 4.3137
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.3438 - loss: 1.1666 - val_accuracy: 0.2778 - val